In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
import boto3
from sagemaker import get_execution_role
from zipfile import ZipFile
import os 

# Get the SageMaker execution role
# sagemaker need permission to write lambda functions

# Get the SageMaker execution role
role = get_execution_role()

# Create a Lambda client using Boto3
client = boto3.client('lambda')



In [31]:
from create_lambda_functions import create_lambda_function,create_deployment_package

### Create SerializeImage function

In [ ]:


# Create 3 Lambda functions
create_lambda_function(client,
                        'serializeImageData',
                       'serialize_image_data',
                       role,
                       10 )

#Upload the serialize_image_data_package.zip
# give more time to the classification image
#create_lambda_function('classifyImage', 'classify_image', role, 60)



### Create deployment package for classifyImage
- Needs sagemaker dependency

In [ ]:
# create .zip deployment package 
# with serialize_image_data.py at the root
# and all the dependencies dirs at the root as well

# Example Usage:
lambda_function_path = 'classify_image.py'
output_zip_path = 'classify_image_package.zip'
#pip install urllib3==1.26.6

dependencies = ['sagemaker',"'urllib3<2.0'"] #"urllib3==1.26.6"

create_deployment_package(lambda_function_path, output_zip_path, dependencies)




In [ ]:


create_lambda_function('filterInferences', 'filter_inferences', role, 10)


### Test Lambda Functions
- call the functions with a payload, a json file

### Step 1: Invoke serializeImageData Lambda and Read the JSON Response

In [34]:
bucket = 'sagemaker-us-east-1-676719617777'
image = "test/bicycle_s_000513.png"

In [ ]:
%%bash

# Invoke the `serializeImageData` Lambda function
aws lambda invoke \
  --function-name serializeImageData \
  --payload '{"s3_bucket": "sagemaker-us-east-1-676719617777", "s3_key": "test/bicycle_s_000513.png"}' \
  serializeImageData_response.json

# View the result
cat serializeImageData_response.json


### Step 2: Use the Base64 Image Data from the JSON Response for the classifyImage Function
Now, extract the base64 image data from the serializeImageData_response.json file and use it as the input for the next Lambda function:

In [ ]:
%%bash 
apt-get install jq


In [48]:
%%bash

# Extract the base64 image data (replace base64_image_data_here with actual value from Step 1)
base64_image_data=$(cat serializeImageData_response.json | jq -r '.body.image_data')

# Pass the base64 image data to classifyImage Lambda function
aws lambda invoke \
  --function-name classifyImage \
  --payload "{\"body\": {\"image_data\": \"$base64_image_data\"}}" \
  classifyImage_response.json

# View the result of classifyImage
cat classifyImage_response.json


bash: line 3: jq: command not found


{
    "StatusCode": 200,
    "FunctionError": "Unhandled",
    "ExecutedVersion": "$LATEST"
}
{"errorMessage": "An error occurred (ValidationError) when calling the InvokeEndpoint operation: 1 validation error detected: Value at 'body' failed to satisfy constraint: Member must not be null", "errorType": "ValidationError", "stackTrace": ["  File \"/var/task/classify_image.py\", line 21, in lambda_handler\n    inferences = predictor.predict(image)\n", "  File \"/var/task/sagemaker/base_predictor.py\", line 212, in predict\n    response = self.sagemaker_session.sagemaker_runtime_client.invoke_endpoint(**request_args)\n", "  File \"/var/task/botocore/client.py\", line 569, in _api_call\n    return self._make_api_call(operation_name, kwargs)\n", "  File \"/var/task/botocore/client.py\", line 1023, in _make_api_call\n    raise error_class(parsed_response, operation_name)\n"]}

### 3. Test for filterInferences Lambda Function
This function filters the inferences based on a confidence threshold.

In [63]:
%%bash
aws lambda invoke \
  --function-name filterInferences \
  --payload '{"inferences": [0.93, 0.87, 0.72]}' \
  filterInferences_response.json

# View the result
cat filterInferences_response.json


{
    "StatusCode": 200,
    "ExecutedVersion": "$LATEST"
}
{"statusCode": 200, "body": "{\"inferences\": [0.93, 0.87, 0.72]}"}

In [ ]:
{
    "StatusCode": 200,
    "FunctionError": "Unhandled",
    "ExecutedVersion": "$LATEST"
}
{"errorMessage": "THRESHOLD_CONFIDENCE_NOT_MET", "errorType": "Exception", "stackTrace": ["  File \"/var/task/filter_inferences.py\", line 21, in lambda_handler\n    raise Exception(\"THRESHOLD_CONFIDENCE_NOT_MET\")\n"]}